In [15]:
import pandas as pd
import numpy as np
%matplotlib inline
from math import *
import operator
import time
pd.options.mode.chained_assignment = None
import math
import os
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
import seaborn as sns
import xgboost as xgb
import datetime
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn import linear_model, decomposition
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [ ]:
sns.distplot(train["Elevation"])

In [ ]:
sns.distplot(train["Horizontal_Distance_To_Hydrology"])

In [ ]:
sns.distplot(train["Vertical_Distance_To_Hydrology"])

In [ ]:
sns.distplot(train["Horizontal_Distance_To_Roadways"])

In [3]:
predictors = train.columns[1:-1]

In [4]:
X_train = train[predictors]
y_train = train[train.columns[-1]]

In [ ]:
svc = SVC(kernel="linear")
#cross_val_score(svc, X_train.as_matrix(), y_train).mean()

In [3]:
"""
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y_train, 2),
              scoring='accuracy')
rfecv.fit(X_train, y_train)
"""



"\nrfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y_train, 2),\n              scoring='accuracy')\nrfecv.fit(X_train, y_train)\n"

In [8]:
X_train.corrwith(y_train)

Elevation                             0.016090
Aspect                                0.008015
Slope                                 0.087722
Horizontal_Distance_To_Hydrology     -0.010515
Vertical_Distance_To_Hydrology        0.075647
Horizontal_Distance_To_Roadways      -0.105662
Hillshade_9am                        -0.010286
Hillshade_Noon                       -0.098905
Hillshade_3pm                        -0.053399
Horizontal_Distance_To_Fire_Points   -0.089389
Wilderness_Area1                     -0.230117
Wilderness_Area2                      0.014994
Wilderness_Area3                      0.122146
Wilderness_Area4                      0.075774
Soil_Type1                            0.015069
Soil_Type2                            0.022627
Soil_Type3                           -0.016393
Soil_Type4                           -0.027816
Soil_Type5                            0.027692
Soil_Type6                            0.006521
Soil_Type7                                 NaN
Soil_Type8   

In [10]:
X_train["Soil_Type15"].unique()

array([0])

In [7]:
X_train.drop("Soil_Type15", axis=1,inplace=True)
X_train.drop("Soil_Type7", axis=1, inplace=True)

### Variance Selection

In [17]:
threshold = 0.90
vt = VarianceThreshold().fit(X_train)
feat_var_threshold = X_train.columns[vt.variances_ > threshold * (1-threshold)]
feat_var_threshold

Index([u'Elevation', u'Aspect', u'Slope', u'Horizontal_Distance_To_Hydrology',
       u'Vertical_Distance_To_Hydrology', u'Horizontal_Distance_To_Roadways',
       u'Hillshade_9am', u'Hillshade_Noon', u'Hillshade_3pm',
       u'Horizontal_Distance_To_Fire_Points', u'Wilderness_Area1',
       u'Wilderness_Area3', u'Wilderness_Area4', u'Soil_Type10'],
      dtype='object')

### Top 20 most important features


In [19]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
feature_imp = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=["importance"])

In [20]:
feat_imp_20 = feature_imp.sort_values("importance", ascending=False).head(20).index
feat_imp_20

Index([u'Elevation', u'Horizontal_Distance_To_Roadways',
       u'Horizontal_Distance_To_Fire_Points',
       u'Horizontal_Distance_To_Hydrology', u'Hillshade_9am',
       u'Vertical_Distance_To_Hydrology', u'Hillshade_3pm',
       u'Wilderness_Area4', u'Hillshade_Noon', u'Aspect', u'Slope',
       u'Soil_Type38', u'Wilderness_Area1', u'Soil_Type10', u'Soil_Type39',
       u'Soil_Type3', u'Wilderness_Area3', u'Soil_Type4', u'Soil_Type30',
       u'Soil_Type40'],
      dtype='object')

### Univariate Selections



In [22]:
X_minmax = MinMaxScaler(feature_range=(0,1)).fit_transform(X_train)
X_scored = SelectKBest(score_func=chi2, k='all').fit(X_minmax, y_train)
feature_scoring = pd.DataFrame({
        'feature': X_train.columns,
        'score': X_scored.scores_
    })
feat_scored_20 = feature_scoring.sort_values('score', ascending=False).head(20)['feature'].values
feat_scored_20




array(['Wilderness_Area4', 'Soil_Type3', 'Soil_Type10', 'Soil_Type38',
       'Wilderness_Area1', 'Soil_Type39', 'Soil_Type40', 'Soil_Type30',
       'Soil_Type29', 'Wilderness_Area3', 'Elevation', 'Soil_Type4',
       'Soil_Type22', 'Soil_Type17', 'Soil_Type13', 'Soil_Type12',
       'Soil_Type23', 'Wilderness_Area2', 'Soil_Type6', 'Soil_Type2'], dtype=object)

### Recursive Feature Elimination

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [25]:
rfe = RFE(LogisticRegression(), 20)
rfe.fit(X_train, y_train)

feature_rfe_scoring = pd.DataFrame({
        'feature': X_train.columns,
        'score': rfe.ranking_
    })

feat_rfe_20 = feature_rfe_scoring[feature_rfe_scoring['score'] == 1]['feature'].values
feat_rfe_20

array(['Wilderness_Area1', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4',
       'Soil_Type5', 'Soil_Type6', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type17', 'Soil_Type22',
       'Soil_Type35', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40'], dtype=object)

### Final Feature Selection

In [26]:
features = np.hstack([
        feat_var_threshold, 
        feat_imp_20,
        feat_scored_20,
        feat_rfe_20
    ])

features = np.unique(features)
print('Final features set:\n')
for f in features:
    print("\t-{}".format(f))

Final features set:

	-Aspect
	-Elevation
	-Hillshade_3pm
	-Hillshade_9am
	-Hillshade_Noon
	-Horizontal_Distance_To_Fire_Points
	-Horizontal_Distance_To_Hydrology
	-Horizontal_Distance_To_Roadways
	-Slope
	-Soil_Type1
	-Soil_Type10
	-Soil_Type11
	-Soil_Type12
	-Soil_Type13
	-Soil_Type17
	-Soil_Type2
	-Soil_Type22
	-Soil_Type23
	-Soil_Type29
	-Soil_Type3
	-Soil_Type30
	-Soil_Type35
	-Soil_Type37
	-Soil_Type38
	-Soil_Type39
	-Soil_Type4
	-Soil_Type40
	-Soil_Type5
	-Soil_Type6
	-Vertical_Distance_To_Hydrology
	-Wilderness_Area1
	-Wilderness_Area2
	-Wilderness_Area3
	-Wilderness_Area4


### Prepare data for final model

In [27]:
X_train = X_train.ix[:, features]



In [29]:
X_test = test.ix[:,features]

In [31]:
clf4 = xgb.XGBClassifier(missing=np.nan, max_depth=7, n_estimators=350, learning_rate=0.03, nthread=4, subsample=0.95, colsample_bytree=0.85, seed=4242)



In [32]:
cross_val_score(clf4, X_train, y_train).mean()

0.77129629629629637

In [33]:
def sub_to_kaggle(clf, X_test):
    result = clf.predict(X_test)
    result2 = pd.DataFrame(result)
    sub = pd.concat((test["Id"],result2),axis=1)
    sub.columns = sample_sub.columns
    sub_file = os.path.join('submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv')
    sub.to_csv(sub_file,index=False)
    print "Good Luck!"

In [34]:
clf4.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.85,
       gamma=0, learning_rate=0.03, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=350, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=4242, silent=True, subsample=0.95)

In [37]:
sub_to_kaggle(clf4, X_test)

Good Luck!


In [36]:
sample_sub = pd.read_csv("./sampleSubmission.csv")

In [40]:
from sklearn.neural_network import MLPClassifier

ImportError: cannot import name MLPClassifier

## Tensorflow

In [44]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
rng = numpy.random

ImportError: cannot import name pywrap_tensorflow

In [55]:
from sklearn.preprocessing import OneHotEncoder

In [56]:
enc = OneHotEncoder()

In [58]:
enc.fit(y_train.reshape(-1,1))

OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [59]:
enc.n_values_

array([8])

In [60]:
enc.feature_indices_

array([0, 8])

In [61]:
y_encho = enc.fit_transform(y_train.reshape(-1,1))

In [62]:
y_encho

<15120x7 sparse matrix of type '<type 'numpy.float64'>'
	with 15120 stored elements in Compressed Sparse Row format>

In [64]:
clf = RandomForestClassifier(n_jobs= 100)

In [69]:
import tensorflow as tf
import numpy as np
from numpy import genfromtxt

# Build Example Data is CSV format, but use Iris data
from sklearn import datasets
from sklearn.model_selection import train_test_split
import sklearn

ImportError: cannot import name pywrap_tensorflow